In [1]:
import numpy as np
import igl
import meshplot as mp
from scipy.spatial.transform import Rotation
import ipywidgets as iw
import time
from deform_ops_chol import Deformer # or from deform_ops import Deformer, if you'd like non-sparse cholesky

## Interactive Version

In [2]:
## Read in a mesh and normalize vertices
mesh_file = 'data/hand.off'
labels_file = 'data/hand.label.npy'

v, f = igl.read_triangle_mesh(mesh_file)
labels = np.load(labels_file).astype(int)
v -= v.min(axis=0)
v /= v.max()

In [3]:
handle_vertex_positions = v.copy()
pos_f_saver = np.zeros((labels.max() + 1, 6)) # saves transformations in a martix 
handle_indices = np.where(labels!=0)[0]
deformer_ = Deformer(v, f, handle_indices)
def pos_f(s,x,y,z, α, β, γ):
    slices = (labels==s)
    r = Rotation.from_euler('xyz', [α, β, γ], degrees=True)
    v_slice = v[slices] + np.array([[x,y,z]])
    center = v_slice.mean(axis=0)
    handle_vertex_positions[slices] = r.apply(v_slice - center) + center
    pos_f_saver[s - 1] = [x,y,z,α,β,γ]
    t0 = time.time()
    v_deformed = pos_f.deformer(handle_vertex_positions)
    p.update_object(vertices = v_deformed)
    t1 = time.time()
    print('FPS', 1/(t1 - t0))
pos_f.deformer = lambda x:x

In [4]:
def widgets_wrapper():
    segment_widget = iw.Dropdown(options=np.arange(labels.max()) + 1)
    translate_widget = {i:iw.FloatSlider(min=-1, max=1, value=0) 
                        for i in 'xyz'}
    rotate_widget = {a:iw.FloatSlider(min=-90, max=90, value=0, step=1) 
                     for a in 'αβγ'}

    def update_seg(*args):
        (translate_widget['x'].value,translate_widget['y'].value,
        translate_widget['z'].value,
        rotate_widget['α'].value,rotate_widget['β'].value,
        rotate_widget['γ'].value) = pos_f_saver[segment_widget.value]
    segment_widget.observe(update_seg, 'value')
    widgets_dict = dict(s=segment_widget)
    widgets_dict.update(translate_widget)
    widgets_dict.update(rotate_widget)
    return widgets_dict

In [5]:
def position_deformer(target_pos):
    '''Fill in this function to change positions'''
    # smooth original mesh (except vertex handles)
    B = deformer_.smooth_mesh(v)
    # smooth the target mesh (except vertex handles)
    B_prime =  deformer_.smooth_mesh(target_pos)
    # encode the details of B into a set of coefficients and tangents
    coeffs, tangents = deformer_.compute_detail_encoding(B)
    # apply the detail encoding to the target mesh
    B_prime_dets = deformer_.apply_detail_encoding(B_prime,coeffs, tangents)
    target_pos = B_prime + B_prime_dets
    return target_pos
pos_f.deformer = position_deformer

In [6]:
p = mp.plot(handle_vertex_positions, f, c=labels)
iw.interact(pos_f,
            **widgets_wrapper())

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5, 0.19…

interactive(children=(Dropdown(description='s', options=(1, 2, 3, 4), value=1), FloatSlider(value=0.0, descrip…

<function __main__.pos_f(s, x, y, z, α, β, γ)>

## Step by Step Implementation

In [14]:
# load the original mesh and get the handle indices
v, f = igl.read_triangle_mesh('data/woody-hi.off')
labels = np.load('data/woody-hi.label.npy').astype(int)
v -= v.min(axis=0)
v /= v.max()
handle_indices = np.where(labels!=0)[0]

mp.plot(v, f, c=labels)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4306930…

In [18]:
# create instance of Deformer
deformer_ = Deformer(v, f, handle_indices)

## Step 1: Smooth the original mesh (except vertex handles)
B = deformer_.smooth_mesh(v)

mp.plot(B, f, c=labels)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4313118…

In [19]:
## Step 2: Deform the smooth mesh
# compute new positions for one of the handles 
new_positions = np.copy(v)
handle_choice = 2 # choose a handle to move
handle_idx = np.where(labels == handle_choice)[0] # indices of handle # handle choice
new_positions[handle_idx] = new_positions[handle_idx] + np.array([0.1, 0.5, 0.5]) # move the handle by 0.5 in each direction
# smooth the mesh with the target mesh positions 
B_prime = deformer_.smooth_mesh(new_positions)

mp.plot(B_prime, f, c=labels)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5313118…

In [20]:
## Step 3: Encode Displacement Vectors (in local frame)
# step 3.1: encoding the details of B
coeffs, tangents = deformer_.compute_detail_encoding(B)
# step 3.2: compute the details of B_prime and add them back 
B_prime_dets = deformer_.apply_detail_encoding(B_prime, coeffs, tangents)
S_prime = B_prime + B_prime_dets

mp.plot(S_prime, f, c=labels)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5306930…

## Example: deforming 4 meshes

If you want to check specific quantities for each mesh, you can change the handle_choice or shift_choice in the array. Just ensure that the values you choose are all consistent in indicies (i.e. the shift value, handle value, label_file and mesh_file are all at the same index within their respective arrays.)

In [21]:
mesh_files = ['data/hand.off', 'data/woody-hi.off', 'data/cactus.off', 'data/bumpy_plane.off']
labels_files = ['data/hand.label.npy', 'data/woody-hi.label.npy', 'data/cactus.label.npy', 'data/bumpy_plane.label.npy']
handle_choice = [2,3,2,1]
shift_choice = [[0,0.5,0], [-0.3,0,0], [0,0.5,0], [0,0,0.5]]

In [22]:
def load_and_deform_mesh(mesh_file, labels_file, handle_choice, shift_choice):
    v, f = igl.read_triangle_mesh(mesh_file)
    labels = np.load(labels_file).astype(int)
    v -= v.min(axis=0)
    v /= v.max()
    new_pos = v.copy()
    handle_indices = np.where(labels!=0)[0]
    handle1_idx = np.where(labels == handle_choice)[0] # indices of handle vertices
    new_pos[handle1_idx] = v[handle1_idx]+np.array(shift_choice)
    deformer_ = Deformer(v, f, handle_indices)
    # smooth original mesh (except vertex handles)
    B = deformer_.smooth_mesh(v)
    # smooth the target mesh (except vertex handles)
    B_prime =  deformer_.smooth_mesh(new_pos)
    # encode the details of B into a set of coefficients and tangents
    coeffs, tangents = deformer_.compute_detail_encoding(B)
    # apply the detail encoding to the target mesh
    B_prime_dets = deformer_.apply_detail_encoding(B_prime,coeffs, tangents)
    S_prime = B_prime + B_prime_dets
    return (labels, f, v, B, B_prime, S_prime)

def plot_mesh(mesh_file, labels_file, handle_choice, shift_choice, one_row=True, wireframe=True):
    labels, f, v, B, B_prime, S_prime = load_and_deform_mesh(mesh_file, labels_file, handle_choice, shift_choice)
    if (one_row):
        p1 = mp.subplot(v, f, c=labels, s=[1,4,0], shading={"wireframe": wireframe})
        p2 = mp.subplot(B, f, c=labels, data=p1, s=[1,4,1], shading={"wireframe": wireframe})
        p3 = mp.subplot(B_prime, f, c=labels, data=p1, s=[1,4,2], shading={"wireframe": wireframe})
        p4 = mp.subplot(S_prime, f, c=labels, data=p1, s=[1,4,3], shading={"wireframe": wireframe})
    else:
        p1 = mp.subplot(v, f, c=labels, s=[2,2,0], shading={"wireframe": wireframe})
        p2 = mp.subplot(B, f, c=labels, data=p1, s=[2,2,1], shading={"wireframe": wireframe})
        p3 = mp.subplot(B_prime, f, c=labels, data=p1, s=[2,2,2], shading={"wireframe": wireframe})
        p4 = mp.subplot(S_prime, f, c=labels, data=p1, s=[2,2,3], shading={"wireframe": wireframe})

### Plot of hand: 
Plotted in a 2x2 for space constraints, but to visualize in one line, change `one_row` to `True`

In [23]:
mesh_index = 0
plot_mesh(mesh_files[mesh_index], labels_files[mesh_index], handle_choice[mesh_index], shift_choice[mesh_index], one_row = False, wireframe = False)

### Plot of woody: 
Plotted in a 2x2 for space constraints, but to visualize in one line, change `one_row` to `True`

In [24]:
mesh_index = 1
plot_mesh(mesh_files[mesh_index], labels_files[mesh_index], handle_choice[mesh_index], shift_choice[mesh_index], one_row = False)

### Plot of cactus: 
Plotted in a 2x2 for space constraints, but to visualize in one line, change `one_row` to `True`

In [25]:
mesh_index = 2
plot_mesh(mesh_files[mesh_index], labels_files[mesh_index], handle_choice[mesh_index], shift_choice[mesh_index], one_row = False)

### Plot of bumpy plane: 
Plotted in a 2x2 for space constraints, but tovisualize in one line, change `one_row` to `True`

In [23]:
mesh_index = 3
plot_mesh(mesh_files[mesh_index], labels_files[mesh_index], handle_choice[mesh_index], shift_choice[mesh_index], one_row = False, wireframe=False)